In [35]:
import numpy as np
import pandas as pd
# from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV


# Read the data

In [2]:
path = '../../../datasets/objective_subjective/'
subjective_file = 'quote.tok.gt9.5000'

In [3]:
f = open(path + subjective_file, 'r', encoding='utf-8', errors='ignore')
subjective_lines = []
for line in f:
    subjective_lines.append(line)
f.close()

print('Subjective:',len(subjective_lines))
subjective_lines[0]

Subjective: 5000


'smart and alert , thirteen conversations about one thing is a small gem . \n'

In [4]:
objective_file = 'plot.tok.gt9.5000'
f = open(path + objective_file, 'r', encoding='utf-8', errors='ignore')
objective_lines = []
for line in f:
    objective_lines.append(line.rstrip())
f.close()

print('Objective:',len(objective_lines))

Objective: 5000


In [5]:
objective_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter'

## Create tf_idf = f(sentence,term)

In [6]:
all_lines = objective_lines + subjective_lines
print(len(all_lines))
all_lines[0]

10000


'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter'

In [8]:
tfidf = TfidfVectorizer(max_df=0.99) # at 0.99, same result as 1: 
sentence_tfidf = tfidf.fit_transform(all_lines)
vocab = tfidf.vocabulary_
if False:
    tfidf_mat = sentence_tfidf
else:
    tfidf_mat = sentence_tfidf.todense()
print(type(tfidf_mat))
print(len(vocab))

<class 'numpy.matrixlib.defmatrix.matrix'>
20893


In [9]:
sorted(vocab)[258:268]

['93',
 '94',
 '95',
 '996',
 '_boogie',
 'aaa',
 'aaliyah',
 'abandon',
 'abandone',
 'abandoned']

In [10]:
tfidf_mat[:2,270:300]

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

## Create Train and CV sets

In [25]:
labels = np.concatenate((np.zeros((5000,), dtype=int), np.ones((5000,), dtype=int)))
print(labels.shape)
labels[4990:5010]

(10000,)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(
    tfidf_mat, labels, test_size=0.2, random_state=0)

In [27]:
print(X_train.shape, X_cv.shape, y_train.shape, y_cv.shape)

(8000, 20893) (2000, 20893) (8000,) (2000,)


## Gradient Boosting

In [31]:
# N_TREES = 300
# LEARN_RATE = 0.2
# MIN_IN_LEAF = 10
# CV score: 0.8285

# 107 trees
# LEARN_RATE = 0.2
# MAX_DEPTH = 8
# MIN_IN_LEAF = 5 #7
# MAX_FEATURES = 'sqrt'
# CV score: 0.8415

# Gradient Boosting Classifier parameters
N_TREES = int(round(np.sqrt(X_train.shape[0]) * 1.2))
# N_TREES = 500
LEARN_RATE = 0.2
MAX_DEPTH = 8
MIN_IN_LEAF = 5 #7
MAX_FEATURES = 'sqrt'
N_TREES

107

In [32]:
print(y_train[0])
print(X_train[0])


1
[[0. 0. 0. ... 0. 0. 0.]]


In [33]:
from sklearn.ensemble import GradientBoostingClassifier

# loss: deviance: logistic log likelihood
gbc = GradientBoostingClassifier(learning_rate=LEARN_RATE, 
                                   n_estimators=N_TREES, 
                                   min_samples_leaf=MIN_IN_LEAF,
                                   max_features=MAX_FEATURES)
gbc.fit(X_train, y_train)
print('Train score:', gbc.score(X_train, y_train))
print('CV score:', gbc.score(X_cv, y_cv))

Train score: 0.863625
CV score: 0.8415


In [44]:
if True:
    grid = {
        'learning_rate': [.01, .05],
        'max_depth': [8, 16],
        'min_samples_leaf': [5],
        'max_features': ['sqrt'],
        'n_estimators': [300],
        'random_state': [0]
    }
else:  # TEST
    grid = {
    'learning_rate': [1],
    'max_depth': [2], 
    'min_samples_leaf': [2],
#     'max_features': ['sqrt', None],
    'n_estimators': [2],
    'random_state': [0]
}
    
# confusion_score = make_scorer(confusion_rmse, greater_is_better=False)

gbc_grid_cv = GridSearchCV(
    GradientBoostingClassifier(), 
    grid,
    cv=4,  # number of folds
    return_train_score=True,
    verbose=1, 
    n_jobs=-1)
gbc_grid_cv.fit(X_train, y_train)

Fitting 4 folds for each of 4 candidates, totalling 16 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed: 21.7min finished


GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'learning_rate': [0.01, 0.05], 'max_depth': [8, 16], 'min_samples_leaf': [5], 'max_features': ['sqrt'], 'n_estimators': [300], 'random_state': [0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [45]:
y_pred = gbc_grid_cv.predict(X_cv)

print(gbc_grid_cv.best_params_)
print(gbc_grid_cv.best_score_)
res_df = pd.DataFrame(gbc_grid_cv.cv_results_)
res_df

{'learning_rate': 0.05, 'max_depth': 16, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'n_estimators': 300, 'random_state': 0}
0.8815


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_max_features,param_min_samples_leaf,param_n_estimators,param_random_state,...,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
0,231.089302,5.176139,1.025364,0.063726,0.01,8,sqrt,5,300,0,...,0.8745,0.857500,0.012140,4,0.889333,0.890167,0.877833,0.889500,0.886708,0.005133
1,359.524472,1.081115,1.536749,0.431736,0.01,16,sqrt,5,300,0,...,0.8770,0.865500,0.008958,3,0.904167,0.904000,0.898500,0.903000,0.902417,0.002305
2,211.131388,3.241846,1.346371,0.529590,0.05,8,sqrt,5,300,0,...,0.8845,0.871875,0.010015,2,0.919333,0.920333,0.916333,0.918667,0.918667,0.001472
3,340.674182,14.592542,0.754131,0.259608,0.05,16,sqrt,5,300,0,...,0.8880,0.881500,0.007492,1,0.953000,0.955333,0.952833,0.952333,0.953375,0.001157


In [ ]:
# save the model
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open('GBC_'+ str(N_TREES) +'_' + str(LEARN_RATE) 
                        +'_' + str(MIN_IN_LEAF) + '_20min.pkl', 'wb'))


## TODO
### Use emotions vectors as input to model
### Use emotions to inspect input and re-label if needed
### See if remove highest frequency words?